In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [2]:
files = glob.glob(os.path.join(os.path.relpath('raw_data', start=os.curdir),'player*csv'))
files

['raw_data\\players13.csv',
 'raw_data\\players14.csv',
 'raw_data\\players15.csv',
 'raw_data\\players16.csv',
 'raw_data\\players17.csv',
 'raw_data\\players18.csv']

In [10]:
for f,filename in enumerate(files):
    temp = pd.DataFrame(pd.read_csv(filename, encoding = "ISO-8859-1"))
    year = filename.split('.')[-2][-2:]
    print('This is loop number {} covering year 20{}'.format(f,year))
    temp['year'] = year
    if f==0:
        df = temp
    else:
        df = pd.concat([df,temp])
print(df.shape)
#df = df.replace('[^A-Za-z\s]+',' ',regex=True)
df[['First','Last']] = df.NAME.str.extract('(?P<First>\S+)\s+(?P<Last>\S+)', expand=True)
df['NAME'] = df.First + ' ' + df.Last
df.drop(['First','Last'],axis=1,inplace=True)
#rplc_dict={'PUP':'','O':'','IR':'','Q':'', 'SUS':''}
#df=df.replace(rplc_dict, regex=True)
df.head(10)

This is loop number 0 covering year 2013
This is loop number 1 covering year 2014
This is loop number 2 covering year 2015
This is loop number 3 covering year 2016
This is loop number 4 covering year 2017
This is loop number 5 covering year 2018
(1800, 17)


,RK,NAME,TEAM,POS,GMS,PASSING_YDS,PASSING_TD,INT,RUSHING_YDS,RUSHING_TD,RECEPTIONS,REC_YDS,REC_TD,FUM LST,PPG,FPTS,year
0,1,Peyton Manning,DEN,QB,16,5477,55,10,-31,1,0,0,0,6,25.6,409.98,13
1,2,Jamaal Charles,KC,RB,15,0,0,0,1287,12,70,693,7,2,25.2,378.00,13
2,3,Drew Brees,NO,QB,16,5162,39,12,52,3,0,0,0,2,22.4,357.68,13
3,4,Matt Forte,CHI,RB,16,0,0,0,1341,9,75,592,3,2,21.1,338.30,13
4,5,LeSean McCoy,PHI,RB,16,0,0,0,1607,9,52,539,2,1,20.7,330.60,13
5,6,Demaryius Thomas,DEN,WR,16,0,0,0,0,0,92,1430,14,0,19.9,319.00,13
6,7,Josh Gordon,CLE,WR,14,0,0,0,88,0,87,1646,9,0,22.5,314.40,13
7,8,Antonio Brown,PIT,WR,16,15,0,0,4,0,110,1499,8,0,19.3,308.90,13
8,9,AJ Green,CIN,WR,16,0,0,0,0,0,98,1426,11,0,19.2,306.60,13
9,10,Brandon Marshall,CHI,WR,16,0,0,0,0,0,100,1295,12,0,19.1,305.50,13


In [11]:
df.sort_values(['FPTS','POS'],inplace=True)
yr = df.groupby('NAME').agg({'year':'max'})
df = df.merge(yr, on = 'NAME',suffixes=['_orig','_last_played'])
print(df.shape)

(1800, 18)


In [12]:
print(df['year_last_played'])
df['retired']= np.where(df['year_last_played'].astype(int)>17,'False','True')
print(df.loc[df['retired']=='True',:].sort_values('FPTS'))

0       13
1       13
2       13
3       18
4       18
5       18
6       16
7       16
8       16
9       16
10      18
11      18
12      18
13      16
14      13
15      16
16      16
17      16
18      16
19      16
20      16
21      16
22      16
23      16
24      18
25      18
26      18
27      13
28      16
29      16
        ..
1770    18
1771    18
1772    18
1773    18
1774    18
1775    18
1776    18
1777    18
1778    18
1779    18
1780    18
1781    18
1782    18
1783    18
1784    18
1785    18
1786    18
1787    18
1788    18
1789    13
1790    18
1791    18
1792    18
1793    18
1794    18
1795    18
1796    18
1797    18
1798    18
1799    18
Name: year_last_played, Length: 1800, dtype: object
       RK              NAME TEAM POS  GMS  PASSING_YDS  PASSING_TD  INT  \
0     300      Da'Rel Scott  NYG  RB    5            0           0    0   
1     299  Junior Hemingway   KC  WR   16            0           0    0   
2     298      Ronnie Brown  LAC  RB   16           

In [13]:
def clean_names(df):
    for ix, row in df.iterrows():
        try:
            df.loc[ix,'NAME'] = ' '.join(row['NAME'].split('\w+')[0:2])
            print(' '.join(row['NAME'].split(' ')[0:2]))
        except KeyError: 
            df.loc[ix,'Name'] = ' '.join(row['Name'].split(' ')[0:2])
    return df

df = clean_names(df)
df = df[df['retired']=='False']

# Format the names to agree with Fantasy Pros
df2 = pd.DataFrame(pd.read_excel(glob.glob(os.path.join(os.path.relpath('raw_data', start=os.curdir),'fantasy*xlsx'))[0]))
#print(df2.columns)
df2 = clean_names(df2)
df = df.merge(df2, how='outer', left_on='NAME', right_on='Name')
print(df.shape)
summary_df = df.groupby(['NAME','POS']).agg({'year_orig':['count'],'FPTS':['mean','std'],'RK':['mean','std'], 'Rank':'mean', 'Avg_pick': 'mean','ADP':'mean'}).round(2)
print(summary_df.columns)

Da'Rel Scott
Junior Hemingway
Ronnie Brown
Jimmy Garoppolo
Jimmy Garoppolo
Jimmy Garoppolo
Harry Douglas
Harry Douglas
Harry Douglas
Harry Douglas
Peyton Barber
Peyton Barber
Peyton Barber
Jeff Janis
Cory Harkey
Tanner McEvoy
Andre Roberts
Andre Roberts
Andre Roberts
Cody Core
Kenneth Farrow
Lance Dunbar
Lance Dunbar
Lance Dunbar
Demetrius Harris
Demetrius Harris
Demetrius Harris
Stanley Havili
Fozzy Whittaker
Fozzy Whittaker
Fozzy Whittaker
Bennie Fowler
Bennie Fowler
Bennie Fowler
Gary Barnidge
Gary Barnidge
Gary Barnidge
Anthony Sherman
Rishard Matthews
Rishard Matthews
Rishard Matthews
Rishard Matthews
Rishard Matthews
Chris Polk
Chris Polk
Chris Polk
Shane Vereen
Shane Vereen
Shane Vereen
Shane Vereen
Shane Vereen
Corey Fuller
Jalin Marshall
Robert Turbin
Robert Turbin
Robert Turbin
Ricardo Louis
Ricardo Louis
Marcedes Lewis
Marcedes Lewis
Marcedes Lewis
Marcedes Lewis
Marcedes Lewis
Kevin White
EJ Manuel
EJ Manuel
EJ Manuel
Matt Asiata
Matt Asiata
Matt Asiata
Matt Asiata
Dion Sim

Coby Fleener
Coby Fleener
Jake Locker
Jake Locker
Austin Hooper
Austin Hooper
Austin Hooper
Nick O'Leary
Khiry Robinson
Khiry Robinson
Zac Stacy
Zac Stacy
Brandon Gibson
Brandon Gibson
Knile Davis
Knile Davis
Maxx Williams
Brian Leonard
Stephen Anderson
Dontrelle Inman
Dontrelle Inman
Dontrelle Inman
Dontrelle Inman
CJ Spiller
CJ Spiller
CJ Spiller
Dalvin Cook
Dalvin Cook
Jordan Thomas
Corey Coleman
Corey Coleman
Stepfan Taylor
Jeff Cumberland
Jeff Cumberland
Riley Cooper
Riley Cooper
Riley Cooper
Daesean Hamilton
Brandon Weeden
Brandon Weeden
Robert Griffin
Robert Griffin
Robert Griffin
Eddie Royal
Eddie Royal
Eddie Royal
Eddie Royal
Zach Pascal
Matt Cassel
Matt Cassel
Ryan Mathews
Ryan Mathews
Ryan Mathews
Ryan Mathews
Dede Westbrook
Dede Westbrook
Owen Daniels
Owen Daniels
Owen Daniels
Jeremy Butler
CJ Anderson
CJ Anderson
CJ Anderson
CJ Anderson
CJ Anderson
Jamaal Charles
Jamaal Charles
Jamaal Charles
Jamaal Charles
Kamar Aiken
Kamar Aiken
Kamar Aiken
Kerwynn Williams
Brock Osweile

Marvin Jones
Marvin Jones
Marvin Jones
Nate Washington
Nate Washington
Nate Washington
Charles Clay
Charles Clay
Charles Clay
Charles Clay
Charles Clay
Allen Robinson
Allen Robinson
Allen Robinson
Allen Robinson
Ameer Abdullah
Ameer Abdullah
Jonathan Stewart
Jonathan Stewart
Jonathan Stewart
Jonathan Stewart
Trent Richardson
Trent Richardson
Robert Woods
Robert Woods
Robert Woods
Robert Woods
Robert Woods
Robert Woods
Tyler Lockett
Tyler Lockett
Tyler Lockett
Tyler Lockett
Sam Bradford
Sam Bradford
Sam Bradford
BenJarvus Green-Ellis
Anthony Miller
Matt Forte
Matt Forte
Matt Forte
Matt Forte
Matt Forte
Leonard Fournette
Leonard Fournette
Jimmy Graham
Jimmy Graham
Jimmy Graham
Jimmy Graham
Jimmy Graham
Jimmy Graham
Marques Colston
Marques Colston
Marques Colston
Heath Miller
Heath Miller
Heath Miller
Isaiah Crowell
Isaiah Crowell
Isaiah Crowell
Isaiah Crowell
Isaiah Crowell
James White
James White
James White
James White
Pierre Thomas
Pierre Thomas
Nick Mullens
Christian Kirk
Lamar Mille

In [14]:
summary_df.reset_index().sort_values(['POS',('Avg_pick','mean'),('FPTS','mean')], ascending=True).reset_index(drop=True)

NAME POS year_orig    FPTS              RK          \
                                   count    mean     std    mean     std   
0            Kyle Juszczyk  FB         5   69.48   20.73  231.80   41.12   
1          Patrick Mahomes  QB         1  417.08     NaN    1.00     NaN   
2           Deshaun Watson  QB         2  250.28  115.15   46.50   53.03   
3            Aaron Rodgers  QB         6  274.50  101.66   42.67   54.14   
4           Baker Mayfield  QB         1  240.10     NaN   42.00     NaN   
5                Matt Ryan  QB         6  282.80   56.52   23.17   16.59   
6             Carson Wentz  QB         3  229.79   46.12   45.00   28.35   
7               Cam Newton  QB         6  295.34   50.61   19.33   14.35   
8           Russell Wilson  QB         6  308.74   33.93   13.83    9.62   
9               Drew Brees  QB         6  310.95   31.97   11.83    7.19   
10              Jared Goff  QB         3  206.25  135.41  100.00  136.01   
11          Jameis Winston  QB         4  232.20   39.48   42.00   20.90   
12           Lamar Jackson  QB         1  157.54     NaN   98.00     NaN   
13      Ben Roethlisberger  QB         6  274.98   41.39   24.83   14.61   
14            Dak Prescott  QB         3  277.80   14.85   20.67    5.13   
15           Philip Rivers  QB         6  275.35   11.89   22.33    4.84   
16               Tom Brady  QB         6  285.00   33.36   20.83   12.29   
17          Mitch Trubisky  QB         2  198.77   90.86   79.50   61.52   
18            Kirk Cousins  QB         5  248.57   89.97   51.60   77.60   
19              Josh Allen  QB         1  208.06     NaN   57.00     NaN   
20         Jimmy Garoppolo  QB         3   57.30   27.18  255.67   60.30   
21             Sam Darnold  QB         1  168.40     NaN   90.00     NaN   
22        Matthew Stafford  QB         6  264.56   28.75   26.83   15.85   
23              Derek Carr  QB         5  230.84   36.65   42.80   19.64   
24             Andy Dalton  QB         6  233.97   39.73   41.33   23.30   
25              Nick Foles  QB         4  137.96   83.03  144.50   80.95   
26            Josh Johnson  QB         1   49.60     NaN  278.00     NaN   
27             CJ Beathard  QB         2   81.89    9.77  203.00   24.04   
28            Jeff Driskel  QB         1   85.12     NaN  201.00     NaN   
29          Brock Osweiler  QB         4  105.66   48.53  178.00   62.59   
..                     ...  ..       ...     ...     ...     ...     ...   
270          Robert Foster  WR         1   99.10     NaN  174.00     NaN   
271          Josh Reynolds  WR         1  100.00     NaN  173.00     NaN   
272            David Moore  WR         1  101.00     NaN  171.00     NaN   
273        Dontrelle Inman  WR         4  101.07   43.69  183.50   54.87   
274           Seth Roberts  WR         4  105.28   10.12  167.25   10.87   
275         Taylor Gabriel  WR         5  106.36   41.10  172.40   61.76   
276  Cordarrelle Patterson  WR         5  106.58   26.33  166.80   34.29   
277            Chris Hogan  WR         5  108.37   12.94  164.00   16.61   
278            Keelan Cole  WR         2  110.95   30.90  158.00   46.67   
279         Danny Amendola  WR         6  111.06   39.71  165.83   65.36   
280        Jermaine Kearse  WR         6  113.36   39.54  161.17   58.01   
281           Josh Doctson  WR         2  114.50    7.50  152.50   12.02   
282         Terrelle Pryor  WR         4  115.02   78.66  178.75  108.47   
283        Martavis Bryant  WR         4  119.68   52.50  159.50   86.25   
284           Torrey Smith  WR         6  121.03   64.81  160.00   87.32   
285           Cole Beasley  WR         6  124.83   39.91  145.67   46.12   
286         Nelson Agholor  WR         4  125.52   64.70  152.75   91.34   
287            Allen Hurns  WR         5  128.28   63.49  150.20   75.34   
288       Antonio Callaway  WR         1  132.30     NaN  133.00     NaN   
289               Ted Ginn  WR         5  135.90   51.20  139.00   78.

In [15]:
summary_df.to_csv(os.path.join(os.path.relpath(os.curdir),'summarized.csv'))

In [18]:
summary_df.sort_values(('ADP','mean')).reset_index(drop=True)

year_orig    FPTS              RK          Rank Avg_pick   ADP
        count    mean     std    mean     std  mean     mean  mean
0           1  385.80     NaN    2.00     NaN   1.0      1.6   1.0
1           2  307.05  110.95   18.50   21.92   2.0      2.4   2.0
2           2  334.30   28.14    5.00    1.41   3.0      3.0   3.0
3           3  285.93   71.67   22.33   25.01   6.0      8.1   4.0
4           6  257.50   79.42   41.83   46.15   4.0      5.9   5.0
5           5  201.14   97.64   84.80   77.76   5.0      6.4   6.0
6           3  288.07  105.31   32.67   29.02   8.0      9.0   7.0
7           1  280.00     NaN   31.00     NaN  13.0     13.3   9.0
8           6  270.98   92.02   40.83   59.46   7.0      8.9  10.0
9           3  276.57   33.75   23.00    6.56  10.0     10.5  11.0
10          5  243.05  100.15   59.20   86.69  11.0     11.3  12.0
11          4  290.50  100.88   33.50   35.93  21.0     21.8  13.0
12          2  244.30   74.39   43.00   29.70  12.0     12.2  14.0
13          5  224.76   44.52   50.80   23.74   9.0      9.9  15.0
14          2  192.35   72.20   76.50   51.62  19.0     19.5  16.0
15          3  255.40   66.01   36.00   28.16  17.0     17.3  17.0
16          1  194.50     NaN   66.00     NaN  15.0     16.2  18.0
17          2  108.70   61.24  167.50   88.39  16.0     16.9  19.0
18          6  335.55   36.11    6.50    5.09  22.0     23.1  20.0
19          1  417.08     NaN    1.00     NaN  37.0     38.7  21.0
20          5  250.66   45.64   36.40   19.53  18.0     18.3  22.0
21          3  247.40   56.45   40.00   28.93  23.0     26.4  23.0
22          5   70.92   23.64  228.00   43.14  26.0     28.8  24.0
23          5  220.14   52.80   57.20   36.49  20.0     20.9  25.0
24          2  175.30   77.64   89.50   79.90  29.0     31.6  26.0
25          6  181.28   58.80   91.50   48.43  31.0     32.1  27.0
26          1  139.40     NaN  119.00     NaN  27.0     28.9  28.0
27          2  182.60  107.62   98.00   84.85  25.0     27.6  29.0
28          2  125.70   64.63  143.50   78.49  30.0     31.7  31.0
29          4  204.35   31.67   61.00   21.95  28.0     29.5  33.0
..        ...     ...     ...     ...     ...   ...      ...   ...
270         1   99.10     NaN  174.00     NaN   NaN      NaN   NaN
271         6  167.62   69.17  105.33   65.51   NaN      NaN   NaN
272         3   87.50   34.19  202.00   59.57   NaN      NaN   NaN
273         4   66.30   25.47  241.50   46.85   NaN      NaN   NaN
274         1   69.40     NaN  231.00     NaN   NaN      NaN   NaN
275         5  221.86   55.11   58.00   39.54   NaN      NaN   NaN
276         4  105.28   10.12  167.25   10.87   NaN      NaN   NaN
277         3  118.57   65.27  165.00   78.82   NaN      NaN   NaN
278         2   63.95   19.73  240.00   49.50   NaN      NaN   NaN
279         4  138.60   36.88  128.75   44.73   NaN      NaN   NaN
280         2   88.25   24.11  198.50   38.89   NaN      NaN   NaN
281         5  106.36   41.10  172.40   61.76   NaN      NaN   NaN
282         2   68.50   27.01  231.50   54.45   NaN      NaN   NaN
283         5  135.90   51.20  139.00   78.21   NaN      NaN   NaN
284         4  115.02   78.66  178.75  108.47   NaN      NaN   NaN
285         5  141.20   34.60  124.80   39.80   NaN      NaN   NaN
286         1   60.20     NaN  256.00     NaN   NaN      NaN   NaN
287         6  121.03   64.81  160.00   87.32   NaN      NaN   NaN
288         1   46.00     NaN  284.00     NaN   NaN      NaN   NaN
289         2   75.75   28.64  219.00   66.47   NaN      NaN   NaN
290         1   48.70     NaN  279.00     NaN   NaN      NaN   NaN
291         2   62.85    3.32  240.00    0.00   NaN      NaN   NaN
292         4  201.65  109.25   96.00  136.32   NaN      NaN   NaN
293         6  110.87   55.16  173.33   68.11   NaN      NaN   NaN
294         3   45.60    6.31  281.33   19.14   NaN      NaN   NaN
295         2   74.70   42.71  225.00   91.92   NaN      NaN   NaN
296         3   70.27   40.82  230.33   68.73   NaN